In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Found existing installation: torch 2.6.0+cu118
Uninstalling torch-2.6.0+cu118:
  Successfully uninstalled torch-2.6.0+cu118
Found existing installation: torchvision 0.21.0+cu118
Uninstalling torchvision-0.21.0+cu118:
  Successfully uninstalled torchvision-0.21.0+cu118
Found existing installation: torchaudio 2.6.0+cu118
Uninstalling torchaudio-2.6.0+cu118:
  Successfully uninstalled torchaudio-2.6.0+cu118
^C
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (6.6 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl (848.7 MB)
Using cached https://download.pytorch.org/wh

In [ ]:
# Install dependencies
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers requests beautifulsoup4 pandas numpy pydrive

import torch
import pandas as pd
import json
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials

# ---- Step 1: Authenticate and Access Google Drive ----
scope = ['https://www.googleapis.com/auth/drive']
service_account_file = "/content/possible-lotus-454715-m2-77512ecfd179.json"  # Change this to your service account JSON key file

creds = ServiceAccountCredentials.from_json_keyfile_name(service_account_file, scope)
gauth = GoogleAuth()
gauth.credentials = creds
drive = GoogleDrive(gauth)

# ---- Step 2: Download JSON File from Google Drive ----
message_json_id = "1TjwNv-rvYe2aghxJVVzmA9-tQXhvn6pv"  # Replace with your actual file ID from Google Drive
json_file_path = "/content/message_1.json"

downloaded_file = drive.CreateFile({'id': message_json_id})
downloaded_file.GetContentFile(json_file_path)

# ---- Step 3: Load JSON Data ----
with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Extract messages from "dummy26769"
sender_name = "dummy26769"
messages = [
    msg["content"] for msg in data["messages"] if msg["sender_name"] == sender_name and "content" in msg
]

# ---- Step 4: Clean Text Data ----
def clean_text(text):
    if isinstance(text, str):
        text = text.encode('latin1').decode('utf-8', 'ignore')
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

cleaned_messages = [clean_text(msg) for msg in messages if msg.strip()]

# ---- Step 5: Load Sentiment Analysis Model ----
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# ---- Step 6: Perform Sentiment Analysis ----
def get_sentiment(text):
    tokens = tokenizer.encode(text, return_tensors='pt', truncation=True, max_length=512)
    result = model(tokens)
    return int(torch.argmax(result.logits)) + 1  # Sentiment score (1-5 scale)

sentiment_scores = [get_sentiment(msg) for msg in cleaned_messages]

# ---- Step 7: Calculate and Print Average Sentiment Score ----
if sentiment_scores:
    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)
    print(f"Average Sentiment Score: {avg_sentiment:.2f}")
else:
    print("No valid messages found for sentiment analysis.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Average Sentiment Score: 2.56


In [ ]:
# ---- Step 8: Save Sentiment Score to JSON File ----
sentiment_data = {"average_sentiment": avg_sentiment}

sentiment_file_path = "/content/sentiment_score.json"

with open(sentiment_file_path, "w") as json_file:
    json.dump(sentiment_data, json_file)

print(f"Sentiment score saved to {sentiment_file_path}")


Sentiment score saved to /content/sentiment_score.json


In [ ]:
file_list = drive.ListFile({'q': "trashed=false"}).GetList()
for file in file_list:
    print(f"File Name: {file['title']}, File ID: {file['id']}")


File Name: sentiment_score.json, File ID: 1GnEtPVR7w0g7QxVija-IidPKx0S9KdJp
File Name: sentiment_score.json, File ID: 1lAqPo26iAc5lrDUKMPXJWLOHt0NZpWSF
File Name: SA_test, File ID: 1LEyFDlteL-x0g_p7F-PlJK6bMe2CQTYj
File Name: message_1.json, File ID: 1TjwNv-rvYe2aghxJVVzmA9-tQXhvn6pv


In [ ]:
file_id = "1GnEtPVR7w0g7QxVija-IidPKx0S9KdJp"
file = drive.CreateFile({'id': file_id})
file.Upload()
file.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})

print(f"🔗 File Link: https://drive.google.com/file/d/{file_id}/view?usp=sharing")


🔗 File Link: https://drive.google.com/file/d/1GnEtPVR7w0g7QxVija-IidPKx0S9KdJp/view?usp=sharing


In [ ]:
from googleapiclient.discovery import build
drive_service = build("drive", "v3", credentials=creds)

print("✅ Google Drive API authenticated successfully!")

✅ Google Drive API authenticated successfully!


In [ ]:
import requests

# Replace with your Project IDX server URL
IDX_SERVER_URL = "https://your-project-id.run.app/receive-link"

# Send the link to IDX
data = {"link": file_link}
response = requests.post(IDX_SERVER_URL, json=data)

print(f"📡 Link sent to IDX: {response.text}")
